In [44]:
! pip install pandas==2.3.3


[notice] A new release of pip is available: 23.2.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [45]:
import pandas as pd
from pathlib import Path

In [46]:
BASE_DIR = Path.cwd().parent      # .../AUTOPRICE-IQ/ETL
FILE1 = BASE_DIR / 'data' / 'leboncoin.csv'     # .../AUTOPRICE-IQ/ETL/data/leboncoin.csv

In [47]:
df_leboncoin = pd.read_csv(FILE1)
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement


In [48]:
import re

In [49]:
allowed = re.compile(r'[^a-zA-Z0-9., ]+')

In [50]:
df_leboncoin["title"] = (
    df_leboncoin["title"]
        .astype(str)
        .apply(lambda x: allowed.sub("", x))
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
)
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement


In [51]:
df_leboncoin["marque"] = (
    df_leboncoin["title"]
        .astype(str)
        .str.split()
        .apply(lambda mots: " ".join(mots[:3]))
)
df_leboncoin[["title", "marque"]].head()

,title,marque
0,Renault Clio 1.0 TCe 90ch Equilibre,Renault Clio 1.0
1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,BMW SERIE 4
2,Audi A3 TFSI,Audi A3 TFSI
3,Lexus RX450h IV 313cv Luxe pack techno,Lexus RX450h IV
4,Mercedes classe a,Mercedes classe a


In [52]:
df_leboncoin['fuel'].value_counts()

fuel
Diesel                  154
Essence                 134
Hybride                  29
Électrique               12
GPL                       4
Hybride Rechargeable      2
Autre                     1
Name: count, dtype: int64

In [53]:
df_leboncoin['gearbox'].value_counts()

gearbox
Manuelle       205
Automatique    131
Name: count, dtype: int64

In [54]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a


In [55]:
def split_location(loc):
    s = str(loc)
    m = re.search(r"(\d{5})", s)
    if not m:
        return pd.Series([s.strip(), None])
    
    ville = s[:m.start()].strip()
    cp = m.group(1)
    return pd.Series([ville, cp])

df_leboncoin[["ville", "code postale"]] = df_leboncoin["location"].apply(split_location)

df_leboncoin[["location", "ville", "code postale"]].head(10)

,location,ville,code postale
0,Metz 57000,Metz,57000
1,Béziers 34500,Béziers,34500
2,Heyrieux 38540,Heyrieux,38540
3,Bouc-Bel-Air 13320,Bouc-Bel-Air,13320
4,Marseille 13012 12e Arrondissement,Marseille,13012
5,Chaponost 69630,Chaponost,69630
6,Chaponost 69630,Chaponost,69630
7,Chaponost 69630,Chaponost,69630
8,Chaponost 69630,Chaponost,69630
9,Chaponost 69630,Chaponost,69630


In [56]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012


In [57]:
df_leboncoin.dropna()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012
...,...,...,...,...,...,...,...,...,...,...,...
331,331,307,1700,2005,283000,Diesel,Manuelle,Charancieu 38490,307,Charancieu,38490
332,332,Bmw 320d e90,13000,2006,150000,Diesel,Automatique,Le Mans 72000,Bmw 320d e90,Le Mans,72000
333,333,Seat 6j 1.2 tdi,2000,2010,110000,Diesel,Manuelle,Saint-Joseph 97212,Seat 6j 1.2,Saint-Joseph,97212
334,334,Fiat 500,6500,2015,91000,Essence,Manuelle,Saint-Génis-des-Fontaines 66740,Fiat 500,Saint-Génis-des-Fontaines,66740


In [58]:
cols = ["price_eur", "year", "kilometers"]

for c in cols:
    df_leboncoin[c] = pd.to_numeric(df_leboncoin[c], errors="coerce")

In [59]:
df_leboncoin.shape

(336, 11)

In [60]:
mask = df_leboncoin["marque"].str.contains(r"[A-Za-z]", regex=True, na=False)

df_leboncoin = df_leboncoin[mask].copy().reset_index(drop=True)

In [61]:
df_leboncoin.shape

(333, 11)

In [62]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012


In [63]:
df_leboncoin["host"] = (df_leboncoin["marque"]
                        .astype(str)
                        .str.split()
                        .apply(lambda x: "".join(x[:1])))
df_leboncoin[["title", "marque", "host"]].head()

,title,marque,host
0,Renault Clio 1.0 TCe 90ch Equilibre,Renault Clio 1.0,Renault
1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,BMW SERIE 4,BMW
2,Audi A3 TFSI,Audi A3 TFSI,Audi
3,Lexus RX450h IV 313cv Luxe pack techno,Lexus RX450h IV,Lexus
4,Mercedes classe a,Mercedes classe a,Mercedes


In [64]:
df_leboncoin.head()

,id,title,price_eur,year,kilometers,fuel,gearbox,location,marque,ville,code postale,host
0,0,Renault Clio 1.0 TCe 90ch Equilibre,15499,2023,23889,Essence,Manuelle,Metz 57000,Renault Clio 1.0,Metz,57000,Renault
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,15490,2017,182000,Diesel,Automatique,Béziers 34500,BMW SERIE 4,Béziers,34500,BMW
2,2,Audi A3 TFSI,12900,2018,124,Essence,Manuelle,Heyrieux 38540,Audi A3 TFSI,Heyrieux,38540,Audi
3,3,Lexus RX450h IV 313cv Luxe pack techno,26000,2017,168500,Hybride,Automatique,Bouc-Bel-Air 13320,Lexus RX450h IV,Bouc-Bel-Air,13320,Lexus
4,4,Mercedes classe a,26000,2021,45144,Diesel,Automatique,Marseille 13012 12e Arrondissement,Mercedes classe a,Marseille,13012,Mercedes


In [65]:
cols_order = [
    "id",
    "title",
    "marque",
    "host",
    "year",
    "kilometers",
    "price_eur",
    "fuel",
    "gearbox",
    "ville",
    "code postale",
    "location"
]
df_leboncoin = df_leboncoin[cols_order]
df_leboncoin.head()

,id,title,marque,host,year,kilometers,price_eur,fuel,gearbox,ville,code postale,location
0,0,Renault Clio 1.0 TCe 90ch Equilibre,Renault Clio 1.0,Renault,2023,23889,15499,Essence,Manuelle,Metz,57000,Metz 57000
1,1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,BMW SERIE 4,BMW,2017,182000,15490,Diesel,Automatique,Béziers,34500,Béziers 34500
2,2,Audi A3 TFSI,Audi A3 TFSI,Audi,2018,124,12900,Essence,Manuelle,Heyrieux,38540,Heyrieux 38540
3,3,Lexus RX450h IV 313cv Luxe pack techno,Lexus RX450h IV,Lexus,2017,168500,26000,Hybride,Automatique,Bouc-Bel-Air,13320,Bouc-Bel-Air 13320
4,4,Mercedes classe a,Mercedes classe a,Mercedes,2021,45144,26000,Diesel,Automatique,Marseille,13012,Marseille 13012 12e Arrondissement


In [69]:
df_leboncoin = df_leboncoin.drop(columns=["id"])
df_leboncoin

,title,marque,host,year,kilometers,price_eur,fuel,gearbox,ville,code postale,location
0,Renault Clio 1.0 TCe 90ch Equilibre,Renault Clio 1.0,Renault,2023,23889,15499,Essence,Manuelle,Metz,57000,Metz 57000
1,BMW SERIE 4 GRAN COUPE F36 420d 190ch BVA8 Lou...,BMW SERIE 4,BMW,2017,182000,15490,Diesel,Automatique,Béziers,34500,Béziers 34500
2,Audi A3 TFSI,Audi A3 TFSI,Audi,2018,124,12900,Essence,Manuelle,Heyrieux,38540,Heyrieux 38540
3,Lexus RX450h IV 313cv Luxe pack techno,Lexus RX450h IV,Lexus,2017,168500,26000,Hybride,Automatique,Bouc-Bel-Air,13320,Bouc-Bel-Air 13320
4,Mercedes classe a,Mercedes classe a,Mercedes,2021,45144,26000,Diesel,Automatique,Marseille,13012,Marseille 13012 12e Arrondissement
...,...,...,...,...,...,...,...,...,...,...,...
328,Twizy 45 cargo sans permis,Twizy 45 cargo,Twizy,2018,17848,6000,Électrique,Automatique,Beauzelle,31700,Beauzelle 31700
329,Bmw 320d e90,Bmw 320d e90,Bmw,2006,150000,13000,Diesel,Automatique,Le Mans,72000,Le Mans 72000
330,Seat 6j 1.2 tdi,Seat 6j 1.2,Seat,2010,110000,2000,Diesel,Manuelle,Saint-Joseph,97212,Saint-Joseph 97212
331,Fiat 500,Fiat 500,Fiat,2015,91000,6500,Essence,Manuelle,Saint-Génis-des-Fontaines,66740,Saint-Génis-des-Fontaines 66740


In [78]:
FILE2 = BASE_DIR / 'data' / 'aramisauto.csv'     # .../AUTOPRICE-IQ/ETL/data/aramisauto.csv

In [79]:
df_aramisauto = pd.read_csv(FILE2)
df_aramisauto.head()

,id,title,price_eur,currency,year,kilometers,fuel,gearbox,condition
0,0,MG MG4,30640.0,EUR,2025.0,10.0,Électrique,Automatique,NaN
1,1,Peugeot 2008,34520.0,EUR,2025.0,707.0,Essence,Automatique,Occasion
2,2,Peugeot 3008,46990.0,EUR,2025.0,1725.0,Électrique,Automatique,Occasion
3,3,NaN,NaN,EUR,NaN,NaN,NaN,NaN,NaN
4,4,Peugeot 2008,27420.0,EUR,2025.0,0.0,Essence,Manuelle,NaN


In [80]:
df_aramisauto = df_aramisauto.drop(columns=["condition", "id", "currency"])
df_aramisauto

,title,price_eur,year,kilometers,fuel,gearbox
0,MG MG4,30640.0,2025.0,10.0,Électrique,Automatique
1,Peugeot 2008,34520.0,2025.0,707.0,Essence,Automatique
2,Peugeot 3008,46990.0,2025.0,1725.0,Électrique,Automatique
3,NaN,NaN,NaN,NaN,NaN,NaN
4,Peugeot 2008,27420.0,2025.0,0.0,Essence,Manuelle
...,...,...,...,...,...,...
235,Peugeot 208,25820.0,2025.0,10.0,Essence,Automatique
236,Citroën C5 Aircross,38400.0,2024.0,10.0,Essence,Automatique
237,Peugeot 3008,NaN,2021.0,64445.0,Essence,Automatique
238,Peugeot 2008,35650.0,2025.0,10.0,Essence,Automatique


In [81]:
df_aramisauto = df_aramisauto.dropna()
df_aramisauto.head()

,title,price_eur,year,kilometers,fuel,gearbox
0,MG MG4,30640.0,2025.0,10.0,Électrique,Automatique
1,Peugeot 2008,34520.0,2025.0,707.0,Essence,Automatique
2,Peugeot 3008,46990.0,2025.0,1725.0,Électrique,Automatique
4,Peugeot 2008,27420.0,2025.0,0.0,Essence,Manuelle
5,Peugeot 5008,45350.0,2025.0,10.0,Essence,Automatique


In [82]:
allowed = re.compile(r'[^a-zA-Z0-9., ]+')

In [83]:
df_aramisauto["title"] = (
    df_aramisauto["title"]
        .astype(str)
        .apply(lambda x: allowed.sub("", x))
        .str.replace(r"\s+", " ", regex=True)
        .str.strip()
)
df_aramisauto.head()

C:\Users\M S I\AppData\Local\Temp\ipykernel_27760\4104805703.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aramisauto["title"] = (


,title,price_eur,year,kilometers,fuel,gearbox
0,MG MG4,30640.0,2025.0,10.0,Électrique,Automatique
1,Peugeot 2008,34520.0,2025.0,707.0,Essence,Automatique
2,Peugeot 3008,46990.0,2025.0,1725.0,Électrique,Automatique
4,Peugeot 2008,27420.0,2025.0,0.0,Essence,Manuelle
5,Peugeot 5008,45350.0,2025.0,10.0,Essence,Automatique


In [84]:
df_aramisauto["marque"] = (
    df_aramisauto["title"]
        .astype(str)
        .str.split()
        .apply(lambda mots: " ".join(mots[:3]))
)
df_aramisauto[["title", "marque"]].head()

C:\Users\M S I\AppData\Local\Temp\ipykernel_27760\2350814441.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aramisauto["marque"] = (


,title,marque
0,MG MG4,MG MG4
1,Peugeot 2008,Peugeot 2008
2,Peugeot 3008,Peugeot 3008
4,Peugeot 2008,Peugeot 2008
5,Peugeot 5008,Peugeot 5008


In [85]:
cols = ["price_eur", "year", "kilometers"]

for c in cols:
    df_aramisauto[c] = pd.to_numeric(df_aramisauto[c], errors="coerce")

C:\Users\M S I\AppData\Local\Temp\ipykernel_27760\343232355.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aramisauto[c] = pd.to_numeric(df_aramisauto[c], errors="coerce")


In [86]:
df_aramisauto

,title,price_eur,year,kilometers,fuel,gearbox,marque
0,MG MG4,30640.0,2025.0,10.0,Électrique,Automatique,MG MG4
1,Peugeot 2008,34520.0,2025.0,707.0,Essence,Automatique,Peugeot 2008
2,Peugeot 3008,46990.0,2025.0,1725.0,Électrique,Automatique,Peugeot 3008
4,Peugeot 2008,27420.0,2025.0,0.0,Essence,Manuelle,Peugeot 2008
5,Peugeot 5008,45350.0,2025.0,10.0,Essence,Automatique,Peugeot 5008
...,...,...,...,...,...,...,...
230,Peugeot 2008,34850.0,2025.0,10.0,Essence,Automatique,Peugeot 2008
232,Renault Captur,35750.0,2025.0,10.0,Essence,Automatique,Renault Captur
235,Peugeot 208,25820.0,2025.0,10.0,Essence,Automatique,Peugeot 208
236,Citron C5 Aircross,38400.0,2024.0,10.0,Essence,Automatique,Citron C5 Aircross


In [87]:
df_aramisauto["year"] = df_aramisauto["year"].astype(int)

C:\Users\M S I\AppData\Local\Temp\ipykernel_27760\1270561482.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_aramisauto["year"] = df_aramisauto["year"].astype(int)


In [88]:
df_aramisauto

,title,price_eur,year,kilometers,fuel,gearbox,marque
0,MG MG4,30640.0,2025,10.0,Électrique,Automatique,MG MG4
1,Peugeot 2008,34520.0,2025,707.0,Essence,Automatique,Peugeot 2008
2,Peugeot 3008,46990.0,2025,1725.0,Électrique,Automatique,Peugeot 3008
4,Peugeot 2008,27420.0,2025,0.0,Essence,Manuelle,Peugeot 2008
5,Peugeot 5008,45350.0,2025,10.0,Essence,Automatique,Peugeot 5008
...,...,...,...,...,...,...,...
230,Peugeot 2008,34850.0,2025,10.0,Essence,Automatique,Peugeot 2008
232,Renault Captur,35750.0,2025,10.0,Essence,Automatique,Renault Captur
235,Peugeot 208,25820.0,2025,10.0,Essence,Automatique,Peugeot 208
236,Citron C5 Aircross,38400.0,2024,10.0,Essence,Automatique,Citron C5 Aircross


In [89]:
mask = df_aramisauto["marque"].str.contains(r"[A-Za-z]", regex=True, na=False)

df_aramisauto = df_aramisauto[mask].copy().reset_index(drop=True)

In [91]:
df_aramisauto["host"] = (df_aramisauto["marque"]
                        .astype(str)
                        .str.split()
                        .apply(lambda x: "".join(x[:1])))
df_aramisauto[["title", "marque", "host"]]

,title,marque,host
0,MG MG4,MG MG4,MG
1,Peugeot 2008,Peugeot 2008,Peugeot
2,Peugeot 3008,Peugeot 3008,Peugeot
3,Peugeot 2008,Peugeot 2008,Peugeot
4,Peugeot 5008,Peugeot 5008,Peugeot
...,...,...,...
149,Peugeot 2008,Peugeot 2008,Peugeot
150,Renault Captur,Renault Captur,Renault
151,Peugeot 208,Peugeot 208,Peugeot
152,Citron C5 Aircross,Citron C5 Aircross,Citron


In [92]:
df_aramisauto["location"] = "Unknow"
df_aramisauto["ville"] = "Unknow"
df_aramisauto["code postale"] = "Unknow"

In [93]:
df_aramisauto.head(3)

,title,price_eur,year,kilometers,fuel,gearbox,marque,host,location,ville,code postale
0,MG MG4,30640.0,2025,10.0,Électrique,Automatique,MG MG4,MG,Unknow,Unknow,Unknow
1,Peugeot 2008,34520.0,2025,707.0,Essence,Automatique,Peugeot 2008,Peugeot,Unknow,Unknow,Unknow
2,Peugeot 3008,46990.0,2025,1725.0,Électrique,Automatique,Peugeot 3008,Peugeot,Unknow,Unknow,Unknow


In [94]:
cols_order = [
    "title",
    "marque",
    "host",
    "year",
    "kilometers",
    "price_eur",
    "fuel",
    "gearbox",
    "ville",
    "code postale",
    "location"
]
df_aramisauto = df_aramisauto[cols_order]
df_aramisauto

,title,marque,host,year,kilometers,price_eur,fuel,gearbox,ville,code postale,location
0,MG MG4,MG MG4,MG,2025,10.0,30640.0,Électrique,Automatique,Unknow,Unknow,Unknow
1,Peugeot 2008,Peugeot 2008,Peugeot,2025,707.0,34520.0,Essence,Automatique,Unknow,Unknow,Unknow
2,Peugeot 3008,Peugeot 3008,Peugeot,2025,1725.0,46990.0,Électrique,Automatique,Unknow,Unknow,Unknow
3,Peugeot 2008,Peugeot 2008,Peugeot,2025,0.0,27420.0,Essence,Manuelle,Unknow,Unknow,Unknow
4,Peugeot 5008,Peugeot 5008,Peugeot,2025,10.0,45350.0,Essence,Automatique,Unknow,Unknow,Unknow
...,...,...,...,...,...,...,...,...,...,...,...
149,Peugeot 2008,Peugeot 2008,Peugeot,2025,10.0,34850.0,Essence,Automatique,Unknow,Unknow,Unknow
150,Renault Captur,Renault Captur,Renault,2025,10.0,35750.0,Essence,Automatique,Unknow,Unknow,Unknow
151,Peugeot 208,Peugeot 208,Peugeot,2025,10.0,25820.0,Essence,Automatique,Unknow,Unknow,Unknow
152,Citron C5 Aircross,Citron C5 Aircross,Citron,2024,10.0,38400.0,Essence,Automatique,Unknow,Unknow,Unknow
